In [ ]:
# Importo bibliotecas escenciales para el analisis del sistema

import numpy as np # Numerical Python es la biblioteca para trabajar con números, matrices y vectores de forma rápida y eficiente
import sympy # Symbolic Python es la biblioteca de matemática simbólica

## CASO ESTÁTICO

### Declaro las variables de INPUTS

In [ ]:
g = 9.81  # ACELERACION GRAVITATORIA [m/s^2]

# MASAS DEL VEHICULO
m_vehiculo_kg = 300 # MASA DEL VEHICULO
m_piloto = 65 # MASA DEL PILOTO
total_mass_kg = m_vehiculo_kg + m_piloto # MASA TOTAL DEL AUTO

# DISTRIBUCION DE PESO
weight_distribution_front_percent = 0.4 # DISTRIBUCION DEL PESO DELANTERO
unsprung_mass_front_corner_kg = 28 # MASA NO SUSPENDIDA EN LA ESQUINA DELANTERA
unsprung_mass_rear_corner_kg = 28 # MASA NO SUSPENDIDA EN LA ESQUINA TRASERA

# --- GEOMETRIA (HARDPOINTS) ---- [X, Y, Z] en Metros ------ el [0;0;0] está en el piso, el eje delantero, y en la mitad transversal
tire_center_FL = [0.0, 0.58675, 0.2525] # Centro de rueda (Referencia del eje)
# Lower Wishbone (LWB)
LWB_chassis_front_FL = [-0.07990, 0.26455, 0.12523] # el front hace referencia al que está por la parte frontal del fsae
LWB_chassis_rear_FL  = [ 0.07990, 0.26455, 0.12523] # el rear hace referencia al que está retrasado respecto a la nariz del fsae
LWB_upright_FL       = [ 0.0,     0.58434, 0.14852] # Single point on upright (LBJ)
# Upper Wishbone (UWB)
UWB_chassis_front_FL = [-0.05435, 0.24249, 0.28878]
UWB_chassis_rear_FL  = [ 0.093, 0.24249, 0.27284]
UWB_upright_FL       = [ 0.0301,  0.55383, 0.35648] # Single point on upright (UBJ)
# Pushrod
pushrod_chassis_FL = [0.0, 0.2, 0.5]
pushrod_upright_FL = [0.0, 0.5, 0.25]
# TIE ROD (BIELA DE DIRECCION)
tierod_chassis_FL = [-0.100, 0.270, 0.160]  # En la cremallera de dirección (Inboard steer point)
tierod_upright_FL = [-0.100, 0.520, 0.160]  # En el portamaza (Outboard steer point)
# PARCHE DE CONTACTO (Mantenemos X e Y del centro de la rueda, pero bajamos Z a 0)
contact_patch_FL = [tire_center_FL[0], tire_center_FL[1], 0.0]

# --- INPUTS DE MATERIAL Y GEOMETRÍA --- (SAE 1026)
material_yield_strength_mpa = 310.0  # Límite elástico (Fluencia)
material_youngs_modulus_gpa = 210.0  # Módulo de Young (Solo para Pandeo)
safety_factor_target = 3.5           # Factor de seguridad deseado

# Geometría del Tubo Actual (Para verificar calculos)
current_OD_mm = 20 # DIAMETRO EXTERIOR DEL TUBO
current_ID_mm = 0 # DIAMETRO INTERIOR DEL TUBO


### Calculo de fuerzas en cada HardPoint

In [ ]:
# ==========================================
# --- CALCULO DE MASAS (PREVIO) ---
# ==========================================

# Total de masa no suspendida
total_unsprung_mass_kg = (2 * unsprung_mass_front_corner_kg) + (2 * unsprung_mass_rear_corner_kg)
# Total de masa supendida
total_suspended_mass_kg = total_mass_kg - total_unsprung_mass_kg
# Total de masa supendida en eje delantero
suspended_mass_front_axle_kg = total_suspended_mass_kg * weight_distribution_front_percent
# Total de masa supendida en eje delantero en una esquina
suspended_mass_front_corner_kg = suspended_mass_front_axle_kg / 2

print(f"Masa suspendida por esquina delantera: {suspended_mass_front_corner_kg:.2f} kg")


# ==========================================
# --- CALCULOS VECTORIALES ---
# ==========================================

def calculate_normalized_vector(point1, point2):
    """Calcula vector unitario desde punto1 hacia punto2"""
    vec = np.array(point2) - np.array(point1)
    magnitude = np.linalg.norm(vec)
    return vec / magnitude if magnitude != 0 else np.array([0.0, 0.0, 0.0])

# Vectores Direccionales (Desde Chasis hacia Upright)
v_LWB_front = calculate_normalized_vector(LWB_chassis_front_FL, LWB_upright_FL)
v_LWB_rear  = calculate_normalized_vector(LWB_chassis_rear_FL, LWB_upright_FL)
v_UWB_front = calculate_normalized_vector(UWB_chassis_front_FL, UWB_upright_FL)
v_UWB_rear  = calculate_normalized_vector(UWB_chassis_rear_FL, UWB_upright_FL)
v_pushrod   = calculate_normalized_vector(pushrod_chassis_FL, pushrod_upright_FL)
v_tierod    = calculate_normalized_vector(tierod_chassis_FL, tierod_upright_FL)

print("\n--- Vectores Direccionales Calculados ---")
print(f"Tie Rod Vector: {v_tierod}")

# ==========================================
# --- EQUILIBRIO DE FUERZAS (SOLVER) ---
# ==========================================

# 1. Definir Incógnitas (Magnitudes de fuerza en los links)
F_LWB_f, F_LWB_r = sympy.symbols('F_LWB_f F_LWB_r')
F_UWB_f, F_UWB_r = sympy.symbols('F_UWB_f F_UWB_r')
F_push, F_tie    = sympy.symbols('F_push F_tie')

# 2. Definir Fuerza de Entrada (Reacción del Suelo)
# La masa suspendida empuja el coche hacia abajo, por lo tanto el suelo empuja la rueda hacia ARRIBA.
# Magnitud = Masa * Gravedad
F_ground_mag = suspended_mass_front_corner_kg * g
F_ground_vector = sympy.Matrix([0, 0, F_ground_mag]) # Vector +Z

print(f"\nFuerza de Reacción del Suelo (Input): {F_ground_mag:.2f} N (+Z)")

# 3. Expresar Fuerzas de los Links como Vectores Simbólicos
# F_vector = Magnitud * Vector_Unitario
F_vec_LWB_f = F_LWB_f * sympy.Matrix(v_LWB_front)
F_vec_LWB_r = F_LWB_r * sympy.Matrix(v_LWB_rear)
F_vec_UWB_f = F_UWB_f * sympy.Matrix(v_UWB_front)
F_vec_UWB_r = F_UWB_r * sympy.Matrix(v_UWB_rear)
F_vec_push  = F_push  * sympy.Matrix(v_pushrod)
F_vec_tie   = F_tie   * sympy.Matrix(v_tierod)

# 4. Ecuaciones de Fuerza (Sum Fx, Fy, Fz = 0)
sum_F = F_ground_vector + F_vec_LWB_f + F_vec_LWB_r + F_vec_UWB_f + F_vec_UWB_r + F_vec_push + F_vec_tie

eq_Fx = sympy.Eq(sum_F[0], 0)
eq_Fy = sympy.Eq(sum_F[1], 0)
eq_Fz = sympy.Eq(sum_F[2], 0)

# 5. Ecuaciones de Momento (Sum Mx, My, Mz = 0)
# Punto de referencia para momentos: Centro de Rueda
ref_point = np.array(tire_center_FL)

# Brazos de palanca (r = Posicion_Aplicacion - Punto_Referencia)
# Para la fuerza del suelo, el punto de aplicación es el parche de contacto
r_ground = sympy.Matrix(np.array(contact_patch_FL) - ref_point)

# Para los links, el punto de aplicación es su hardpoint en el upright
r_LWB = sympy.Matrix(np.array(LWB_upright_FL) - ref_point)
r_UWB = sympy.Matrix(np.array(UWB_upright_FL) - ref_point)
r_push = sympy.Matrix(np.array(pushrod_upright_FL) - ref_point)
r_tie = sympy.Matrix(np.array(tierod_upright_FL) - ref_point)

# Momentos individuales (r cruz F)
M_ground = r_ground.cross(F_ground_vector)
M_LWB_f = r_LWB.cross(F_vec_LWB_f)
M_LWB_r = r_LWB.cross(F_vec_LWB_r)
M_UWB_f = r_UWB.cross(F_vec_UWB_f)
M_UWB_r = r_UWB.cross(F_vec_UWB_r)
M_push  = r_push.cross(F_vec_push)
M_tie   = r_tie.cross(F_vec_tie)

# Suma de Momentos
sum_M = M_ground + M_LWB_f + M_LWB_r + M_UWB_f + M_UWB_r + M_push + M_tie

eq_Mx = sympy.Eq(sum_M[0], 0)
eq_My = sympy.Eq(sum_M[1], 0)
eq_Mz = sympy.Eq(sum_M[2], 0)

# 6. Resolver Sistema (6 Ecuaciones, 6 Incógnitas)
unknowns = [F_LWB_f, F_LWB_r, F_UWB_f, F_UWB_r, F_push, F_tie]
system_equations = [eq_Fx, eq_Fy, eq_Fz, eq_Mx, eq_My, eq_Mz]

print("\nResolviendo sistema de ecuaciones...")
solution = sympy.solve(system_equations, unknowns)

# ==========================================
# RESULTADOS
# ==========================================

if solution:
    print("\n=== RESULTADOS DE FUERZAS EN LOS BRAZOS ===")
    print("Signo (+): Compresión (Brazo empuja al upright)")
    print("Signo (-): Tensión (Brazo tira del upright)")

    # Función auxiliar para imprimir
    def print_force(name, symbol):
        val = float(solution[symbol].evalf())
        print(f"{name}: {val:.2f} N")

    print_force("LWB Front", F_LWB_f)
    print_force("LWB Rear ", F_LWB_r)
    print_force("UWB Front", F_UWB_f)
    print_force("UWB Rear ", F_UWB_r)
    print_force("Pushrod  ", F_push)
    print_force("Tie Rod  ", F_tie)

    # Análisis rápido del Momento generado por Scrub Radius (Teórico)
    # Mostramos el momento que genera el suelo antes de ser compensado por la suspensión
    m_ground_val = [float(x) for x in M_ground]
    print(f"\nMomento generado solo por la carga del suelo (Input Moment): {m_ground_val} N.m")
    if m_ground_val[0] == 0 and m_ground_val[1] == 0:
        print("(Es cero porque la carga es vertical pura y está alineada en X/Y con el centro)")

else:
    print("No se encontró solución. Revisa si hay puntos colineales o redundantes.")

Masa suspendida por esquina delantera: 50.60 kg

--- Vectores Direccionales Calculados ---
Tie Rod Vector: [0. 1. 0.]

Fuerza de Reacción del Suelo (Input): 496.39 N (+Z)

Resolviendo sistema de ecuaciones...

=== RESULTADOS DE FUERZAS EN LOS BRAZOS ===
Signo (+): Compresión (Brazo empuja al upright)
Signo (-): Tensión (Brazo tira del upright)
LWB Front: -223.82 N
LWB Rear : -234.14 N
UWB Front: -37.64 N
UWB Rear : -37.33 N
Pushrod  : 698.04 N
Tie Rod  : -22.11 N

Momento generado solo por la carga del suelo (Input Moment): [0.0, 0.0, 0.0] N.m
(Es cero porque la carga es vertical pura y está alineada en X/Y con el centro)


### CÁLCULO DE ESFUERZOS Y DIMENSIONAMIENTO

In [ ]:
# ==========================================
# 7. CÁLCULO DE ESFUERZOS Y DIMENSIONAMIENTO
# ==========================================
print("\n" + "="*50)
print(" ANÁLISIS DE RESISTENCIA DE MATERIALES")
print("="*50)

# --- FUNCIONES AUXILIARES ---

def calculate_length(p1, p2):
    return np.linalg.norm(np.array(p2) - np.array(p1))

def get_cross_section_area(od_mm, id_mm):
    return (np.pi / 4) * (od_mm**2 - id_mm**2)

def get_moment_of_inertia(od_mm, id_mm):
    # Momento de inercia de area (I) para tubo circular
    return (np.pi / 64) * (od_mm**4 - id_mm**4)

def analyze_member(name, force_val, p1, p2):
    length_m = calculate_length(p1, p2)
    length_mm = length_m * 1000

    # 1. Área Actual
    area_mm2 = get_cross_section_area(current_OD_mm, current_ID_mm)
    if area_mm2 <= 0:
        print(f"Error: Área 0 en {name}. Revisa diámetros.")
        return

    # 2. Esfuerzo Axial (Von Mises en barras de dos fuerzas es igual al axial)
    # Stress = Force / Area. (N / mm^2 = MPa)
    axial_stress_mpa = abs(force_val) / area_mm2

    # 3. Factor de Seguridad a la Fluencia (Yielding)
    fs_yield = material_yield_strength_mpa / axial_stress_mpa if axial_stress_mpa > 0 else 999.9

    # 4. Cálculo de Diámetro Mínimo (Para barra maciza, ID=0)
    # Area_req = Force * SF / Yield
    # D_min = sqrt(4 * Area_req / pi)
    area_req_mm2 = (abs(force_val) * safety_factor_target) / material_yield_strength_mpa
    min_solid_diameter_mm = np.sqrt((4 * area_req_mm2) / np.pi)

    # 5. Análisis de Pandeo (Buckling) - SOLO SI ESTÁ EN COMPRESIÓN
    # Convención anterior: Positivo = Compresión
    buckling_info = ""
    is_compression = force_val > 0

    if is_compression:
        # Carga Crítica de Euler: P_cr = (pi^2 * E * I) / (K*L)^2
        # Asumimos extremos articulados (Rod ends), K = 1.0
        I_mm4 = get_moment_of_inertia(current_OD_mm, current_ID_mm)
        E_mpa = material_youngs_modulus_gpa * 1000

        P_crit_N = (np.pi**2 * E_mpa * I_mm4) / (length_mm**2)
        fs_buckling = P_crit_N / force_val

        buckling_status = "OK" if fs_buckling > safety_factor_target else "FALLO (PANDEO)"
        buckling_info = f" | FS Pandeo: {fs_buckling:.2f} ({buckling_status})"
    else:
        buckling_info = " | (Tracción -> No Pandea)"

    # --- IMPRIMIR RESULTADOS ---
    print(f"\n>> {name.upper()}")
    print(f"   Fuerza: {force_val:.2f} N | Longitud: {length_mm:.1f} mm")
    print(f"   Esfuerzo Von Mises: {axial_stress_mpa:.2f} MPa")
    print(f"   FS Fluencia: {fs_yield:.2f}{buckling_info}")
    print(f"   Diam. Mínimo (Macizo) para FS={safety_factor_target}: {min_solid_diameter_mm:.2f} mm")

# --- EJECUCIÓN DEL ANÁLISIS ---

# Extraer valores float de la solución de SymPy
val_LWB_f = float(solution[F_LWB_f].evalf())
val_LWB_r = float(solution[F_LWB_r].evalf())
val_UWB_f = float(solution[F_UWB_f].evalf())
val_UWB_r = float(solution[F_UWB_r].evalf())
val_push  = float(solution[F_push].evalf())
val_tie   = float(solution[F_tie].evalf())

print(f"Inputs: OD={current_OD_mm}mm, ID={current_ID_mm}mm")
print(f"Material: Yield={material_yield_strength_mpa} MPa, E={material_youngs_modulus_gpa} GPa")

# Analizar cada barra con sus coordenadas correspondientes
analyze_member("LWB Front", val_LWB_f, LWB_chassis_front_FL, LWB_upright_FL)
analyze_member("LWB Rear ", val_LWB_r, LWB_chassis_rear_FL,  LWB_upright_FL)
analyze_member("UWB Front", val_UWB_f, UWB_chassis_front_FL, UWB_upright_FL)
analyze_member("UWB Rear ", val_UWB_r, UWB_chassis_rear_FL,  UWB_upright_FL)
analyze_member("Pushrod  ", val_push,  pushrod_chassis_FL,   pushrod_upright_FL)
analyze_member("Tie Rod  ", val_tie,   tierod_chassis_FL,    tierod_upright_FL)


 ANÁLISIS DE RESISTENCIA DE MATERIALES
Inputs: OD=20mm, ID=0mm
Material: Yield=310.0 MPa, E=210.0 GPa

>> LWB FRONT
   Fuerza: -223.82 N | Longitud: 330.4 mm
   Esfuerzo Von Mises: 0.71 MPa
   FS Fluencia: 435.13 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 1.79 mm

>> LWB REAR 
   Fuerza: -234.14 N | Longitud: 330.4 mm
   Esfuerzo Von Mises: 0.75 MPa
   FS Fluencia: 415.95 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 1.83 mm

>> UWB FRONT
   Fuerza: -37.64 N | Longitud: 329.6 mm
   Esfuerzo Von Mises: 0.12 MPa
   FS Fluencia: 2587.25 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 0.74 mm

>> UWB REAR 
   Fuerza: -37.33 N | Longitud: 328.5 mm
   Esfuerzo Von Mises: 0.12 MPa
   FS Fluencia: 2608.72 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 0.73 mm

>> PUSHROD  
   Fuerza: 698.04 N | Longitud: 390.5 mm
   Esfuerzo Von Mises: 2.22 MPa
   FS Fluencia: 139.52 | FS Pandeo: 152.92 (OK)
   Diam. Mínimo (Macizo) para 

# CASO DINÁMICO

### Declaro las variables de INPUTS para el caso dinámico

In [ ]:
# --- INPUTS DE MANIOBRA ---
g_braking   = 1.7  # G's de frenado
g_cornering = 2.0  # G's de curva
g_bump      = 3  # G's de impacto vertical (1.0 = Suelo liso, 3.0 = Bache violento)
# NOTA: Usualmente se analiza Transferencia de Carga (Suelo liso) O Bache puro.
# Si pones 3G de bump + transferencia, estás simulando caer de un salto frenando (caso destructivo).


# --- INPUTS GEOMÉTRICOS DEL VEHÍCULO (Necesarios para transferencia) ---
CG_height_m = 0.35     # Altura del Centro de Gravedad (Estimado FSAE)
wheelbase_m = 1.65     # Distancia entre ejes
track_width_front_m = 1.20 # Ancho de vía (Trocha) delantero


# Coeficientes de Fricción
mu_tire_long = 1
mu_tire_lat  = 1

### ANÁLISIS DINÁMICO AVANZADO

In [ ]:
# ==============================================================================
# ANÁLISIS DINÁMICO AVANZADO (CON TRANSFERENCIA DE CARGA REAL)
# ==============================================================================
print("\n" + "#"*60)
print("   INICIO DEL ANÁLISIS DINÁMICO (FRENADA + CURVA + BUMP)")
print("#"*60)

# --- CÁLCULO DE TRANSFERENCIA DE CARGA ---

# Transferencia Longitudinal (Frenado)
# Delta_Fz_long = (Masa_Total * a_x * h_cg) / Wheelbase
weight_transfer_long_kg = (total_mass_kg * g_braking * CG_height_m) / wheelbase_m

# La carga pasa del eje trasero al delantero.
# Carga original en eje delantero:
static_front_axle_load_kg = suspended_mass_front_axle_kg + (2 * unsprung_mass_front_corner_kg)
# Nueva carga en eje delantero (Estática + Transferencia):
dynamic_front_axle_load_kg = static_front_axle_load_kg + weight_transfer_long_kg

print(f"\n--- TRANSFERENCIA DE CARGA ---")
print(f"Transferencia Longitudinal (al eje delantero): +{weight_transfer_long_kg:.2f} kg")

# 2. Transferencia Lateral (Curva)
# Delta_Fz_lat = (Masa_Eje_Delantero_Dinamica * a_y * h_cg) / Track_Width
# Nota: Esto es una aproximación cinemática. En realidad depende de la rigidez al rolido (LLTD),
# pero asumimos que el chasis es infinitamente rígido para el "peor caso" geométrico.
weight_transfer_lat_kg = (dynamic_front_axle_load_kg * g_cornering * CG_height_m) / track_width_front_m

print(f"Transferencia Lateral (a la rueda externa):    +{weight_transfer_lat_kg:.2f} kg")

# 3. Carga Vertical Total en la Rueda Delantera Externa (FL)
# Masa en la rueda = (Masa Eje / 2) + Transferencia Lateral
mass_on_FL_corner_dynamic = (dynamic_front_axle_load_kg / 2) + weight_transfer_lat_kg

# Convertimos a Fuerza (Newtons) y aplicamos el factor de BUMP (si golpea un bache cargada)
Fz_dynamic = mass_on_FL_corner_dynamic * g * g_bump

print(f"\n>> MASA TOTAL EN RUEDA FL (Cargada): {mass_on_FL_corner_dynamic:.2f} kg")
if g_bump > 1.0:
    print(f">> APLICANDO BUMP ({g_bump}G)...")

# --- FUERZAS EN EL SUELO ---
# Fx y Fy dependen de la carga vertical dinámica (Fz).
# El neumático puede dar F = mu * Fz
Fx_dynamic = -(Fz_dynamic * (g_braking/g_bump)) if g_bump > 0 else 0
# Nota: Dividimos por g_bump porque la fricción depende de la carga,
# pero g_braking es la aceleración del chasis, no del bache.
# SIMPLIFICACION: Usaremos la capacidad máxima del neumático bajo esa carga.

# Recalculamos Fx y Fy basados en la carga Fz real que tiene la rueda ahora:
# (Limitado por lo que pediste de 1.7G y 2G, o por la fricción física)
Fx_req = mass_on_FL_corner_dynamic * g * g_braking # Fuerza necesaria para frenar esa masa
Fy_req = mass_on_FL_corner_dynamic * g * g_cornering # Fuerza necesaria para girar esa masa

# Chequeo de adherencia (toma el valor minimo de los dos sino tomaria fuerzas de 10g los cuales no tenes por adherencia)
Fx_dynamic = -min(Fx_req, Fz_dynamic * mu_tire_long)
Fy_dynamic = min(Fy_req, Fz_dynamic * mu_tire_lat)

print(f"\n--- FUERZAS FINALES EN PARCHE DE CONTACTO ---")
print(f"Fz (Vertical): {Fz_dynamic:.2f} N")
print(f"Fx (Frenado):  {Fx_dynamic:.2f} N")
print(f"Fy (Lateral):  {Fy_dynamic:.2f} N")

# Definir el Vector de Fuerza Dinámica
F_ground_vector_dyn = sympy.Matrix([Fx_dynamic, Fy_dynamic, Fz_dynamic])


# ==============================================================================
# 9. RESOLUCIÓN DEL SISTEMA DE ECUACIONES DINÁMICO
# ==============================================================================

# Recalculamos el Momento que genera el suelo
# Ahora Fx y Fy NO son cero, por lo que r_ground X F_ground generará momentos grandes
M_ground_dyn = r_ground.cross(F_ground_vector_dyn)

print(f"\nMomentos generados por el suelo (Scrub Radius / Caster Trail):")
print(f"Mx (Roll moment):  {float(M_ground_dyn[0]):.2f} N.m")
print(f"My (Drive torque): {float(M_ground_dyn[1]):.2f} N.m")
print(f"Mz (Align torque): {float(M_ground_dyn[2]):.2f} N.m")

# Actualizamos las ecuaciones de Equilibrio
# Suma de Fuerzas
sum_F_dyn = F_ground_vector_dyn + F_vec_LWB_f + F_vec_LWB_r + F_vec_UWB_f + F_vec_UWB_r + F_vec_push + F_vec_tie

# Suma de Momentos
sum_M_dyn = M_ground_dyn + M_LWB_f + M_LWB_r + M_UWB_f + M_UWB_r + M_push + M_tie

# Sistema de Ecuaciones
eq_Fx_dyn = sympy.Eq(sum_F_dyn[0], 0)
eq_Fy_dyn = sympy.Eq(sum_F_dyn[1], 0)
eq_Fz_dyn = sympy.Eq(sum_F_dyn[2], 0)
eq_Mx_dyn = sympy.Eq(sum_M_dyn[0], 0)
eq_My_dyn = sympy.Eq(sum_M_dyn[1], 0)
eq_Mz_dyn = sympy.Eq(sum_M_dyn[2], 0)

system_equations_dyn = [eq_Fx_dyn, eq_Fy_dyn, eq_Fz_dyn, eq_Mx_dyn, eq_My_dyn, eq_Mz_dyn]

solution_dyn = sympy.solve(system_equations_dyn, unknowns)

print("")
print("Fx dinamica: ", Fx_dynamic)
print("Fy dinamica: ", Fy_dynamic)
print("Fz dinamica: ", Fz_dynamic)


############################################################
   INICIO DEL ANÁLISIS DINÁMICO (FRENADA + CURVA + BUMP)
############################################################

--- TRANSFERENCIA DE CARGA ---
Transferencia Longitudinal (al eje delantero): +131.62 kg
Transferencia Lateral (a la rueda externa):    +168.48 kg

>> MASA TOTAL EN RUEDA FL (Cargada): 312.89 kg
>> APLICANDO BUMP (3G)...

--- FUERZAS FINALES EN PARCHE DE CONTACTO ---
Fz (Vertical): 9208.34 N
Fx (Frenado):  -5218.06 N
Fy (Lateral):  6138.89 N

Momentos generados por el suelo (Scrub Radius / Caster Trail):
Mx (Roll moment):  1550.07 N.m
My (Drive torque): 1317.56 N.m
Mz (Align torque): 0.00 N.m

Fx dinamica:  -5218.06063409091
Fy dinamica:  6138.894863636365
Fz dinamica:  9208.342295454548


### CÁLCULO DE ESFUERZOS Y DIMENSIONAMIENTO

In [ ]:
# ==============================================================================
# RESULTADOS DE ESFUERZOS (STRESS) Y DIMENSIONAMIENTO DINÁMICO
# ==============================================================================

if solution_dyn:
    print("\n" + "="*50)
    print(" RESULTADOS: CASO EXTREMO (3G BUMP + 1.7G BRAKE + 2G LAT)")
    print("="*50)

    # Extraer valores
    val_LWB_f_dyn = float(solution_dyn[F_LWB_f].evalf())
    val_LWB_r_dyn = float(solution_dyn[F_LWB_r].evalf())
    val_UWB_f_dyn = float(solution_dyn[F_UWB_f].evalf())
    val_UWB_r_dyn = float(solution_dyn[F_UWB_r].evalf())
    val_push_dyn  = float(solution_dyn[F_push].evalf())
    val_tie_dyn   = float(solution_dyn[F_tie].evalf())

    # Reutilizamos la función analyze_member definida anteriormente

    analyze_member("LWB Front (Dyn)", val_LWB_f_dyn, LWB_chassis_front_FL, LWB_upright_FL)
    analyze_member("LWB Rear (Dyn) ", val_LWB_r_dyn, LWB_chassis_rear_FL,  LWB_upright_FL)
    analyze_member("UWB Front (Dyn)", val_UWB_f_dyn, UWB_chassis_front_FL, UWB_upright_FL)
    analyze_member("UWB Rear (Dyn) ", val_UWB_r_dyn, UWB_chassis_rear_FL,  UWB_upright_FL)
    analyze_member("Pushrod (Dyn)  ", val_push_dyn,  pushrod_chassis_FL,   pushrod_upright_FL)
    analyze_member("Tie Rod (Dyn)  ", val_tie_dyn,   tierod_chassis_FL,    tierod_upright_FL)

else:
    print("Error: No se encontró solución para el caso dinámico.")


 RESULTADOS: CASO EXTREMO (3G BUMP + 1.7G BRAKE + 2G LAT)

>> LWB FRONT (DYN)
   Fuerza: 8163.70 N | Longitud: 330.4 mm
   Esfuerzo Von Mises: 25.99 MPa
   FS Fluencia: 11.93 | FS Pandeo: 18.26 (OK)
   Diam. Mínimo (Macizo) para FS=3.5: 10.83 mm

>> LWB REAR (DYN) 
   Fuerza: -28191.58 N | Longitud: 330.4 mm
   Esfuerzo Von Mises: 89.74 MPa
   FS Fluencia: 3.45 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 20.13 mm

>> UWB FRONT (DYN)
   Fuerza: -6750.11 N | Longitud: 329.6 mm
   Esfuerzo Von Mises: 21.49 MPa
   FS Fluencia: 14.43 | (Tracción -> No Pandea)
   Diam. Mínimo (Macizo) para FS=3.5: 9.85 mm

>> UWB REAR (DYN) 
   Fuerza: 9624.63 N | Longitud: 328.5 mm
   Esfuerzo Von Mises: 30.64 MPa
   FS Fluencia: 10.12 | FS Pandeo: 15.68 (OK)
   Diam. Mínimo (Macizo) para FS=3.5: 11.76 mm

>> PUSHROD (DYN)  
   Fuerza: 13841.65 N | Longitud: 390.5 mm
   Esfuerzo Von Mises: 44.06 MPa
   FS Fluencia: 7.04 | FS Pandeo: 7.71 (OK)
   Diam. Mínimo (Macizo) para FS=3.5: 14.11 